# Linear Models

# Explain the '<' pattern (article+data)
# Build geoapp where you can increase a radius centered at IU (Iowa State University), then see the predicted house prices + actual house prices. Once click on the house, can see the difference explained by examining that house's shap value! 
# Use CatBoost's Shap value to build a Dashboard

# Blend predictions??


In [1]:
KAGGLE=False # set to True if using the Kaggle version of the dataset

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import sklearn.preprocessing

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn import preprocessing
#from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold,KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [3]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)

The data has been pre-processed

In [4]:
def load_data(tree=None):
    if KAGGLE:
        train = pd.read_csv('data/train.csv')
        test = pd.read_csv('data/test.csv')
        num_train=np.shape(train)[0]
        num_test=np.shape(test)[0]
        alldata=train.append(test, ignore_index=True)
    else:
        alldata=pd.read_csv('./data/Ames_HousePriceLocsDist.csv')#, index_col=0) #already cleaned (see work in appendix files)
    return alldata

alldata=load_data()

In [5]:
nums = [col for col in alldata.columns if alldata.dtypes[col] != 'object']
cats = [col for col in alldata.columns if alldata.dtypes[col] == 'object']
alldata=pd.get_dummies(alldata).reset_index(drop=True)

In [6]:
alldata['LogPrice']=np.log1p(alldata.SalePrice)

In [8]:
alldata.distToUI.head()

0    0.620685
1    0.890758
2    0.890758
3    1.702803
4    1.909280
Name: distToUI, dtype: float64

In [9]:
def split_into_train_test(alldata):
    y=alldata['LogPrice']
    X=alldata.drop(['SalePrice','LogPrice'],axis=1)
    from sklearn.model_selection import train_test_split
    if KAGGLE:
        return split_train_test(alldata=alldata,num_train=num_train)
    else:
        return train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = split_into_train_test(alldata)  

The sale price appears to be skewed to the right

In [10]:
# from sklearn.pipeline import make_pipeline 
# from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# pipe = make_pipeline(OneHotEncoder(),StandardScaler(),PolynomialFeatures(3),LinearRegression())
# run_model(pipe,X_train,y_train,X_test,y_test)

In [11]:
def run_model(model,X_train,y_train,X_test,y_test):
    
    # Fit Model
    model.fit(X_train,y_train)
    
    # Get Metrics
    
    preds = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test,preds))
    print(f'RMSE on Test: {rmse}')
    r2 = r2_score(y_test, preds)
    print(f'R^2 on Test: {r2}')

    return model

### Without log price

In [11]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
run_model(model_lr,X_train,y_train,X_test,y_test)

RMSE : 21682.499039879214
R^2 : 0.9274217627102791


LinearRegression()

### With log price

In [18]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
run_model(model_lr,X_train,y_train,X_test,y_test)

RMSE : 0.10506923188036138
R^2 : 0.9292558726935111


LinearRegression()

## Size-related vars

In [12]:
size_related = alldata.filter(regex='SF$|Area$')


In [13]:
size_related.head()

,GrLivArea,LotArea,MasVnrArea,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea
0,856,7890,0.0,618.0,856.0,856,0,0,399.0,0,0,0
1,1049,4235,149.0,104.0,1049.0,1049,0,0,266.0,0,105,0
2,1049,4235,149.0,104.0,1049.0,1049,0,0,266.0,0,105,0
3,1001,6060,0.0,100.0,837.0,1001,0,0,216.0,154,0,0
4,1039,8146,0.0,405.0,405.0,717,322,0,281.0,0,0,0


In [14]:
size_related.columns

Index(['GrLivArea', 'LotArea', 'MasVnrArea', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'PoolArea'],
      dtype='object')

In [ ]:
#feature_imp_cols=

In [ ]:
#

In [ ]:
pd.concat([size_related, alldata['LogPrice']], axis=1)

In [23]:
### Pipeline

In [ ]:
pipe = make_pipeline(StandardScaler(),PolynomialFeatures(3),LinearRegression())
run_model(pipe,X_train,y_train,X_test,y_test)